# Wingsail Controller Prototype

In [ ]:
# Install Python dependencies
!pip3 install numpy scipy matplotlib


In [ ]:
# Imports
%matplotlib inline
import numpy as np
import scipy
import matplotlib.pyplot as plt


## Function `compute_reynolds_number` Implementation

It takes the apparent wind speed and the chord width to computes the Reynold's number.

In [ ]:
CHORD_WIDTH_MAIN_SAIL = 0.14  # meters, trim tab chord width is not included
KINEMATIC_VISCOSITY = 0.000014207  # {m^2 / s at 10degC} and air density at 1.225 {kg / m^3}


def compute_reynolds_number(apparent_wind_speed):
    reynolds_number = (apparent_wind_speed * CHORD_WIDTH_MAIN_SAIL) / KINEMATIC_VISCOSITY
    return reynolds_number


## Function `compute_angle_of_attack` Implementation

It takes the Reynolds number and uses a lookup table to find the angle of attack.

In [ ]:
def compute_angle_of_attack(reynolds_number, look_up_table):
    desired_alpha = np.interp(reynolds_number, look_up_table[:, 0], look_up_table[:, 1])
    return desired_alpha


## Function `compute_trim_tab_angle` Implementation

It takes the desired alpha value and computes the resulting trim tab angle.

In [ ]:
def compute_trim_tab_angle(desired_alpha, wind_direction, boat_direction):
    # additional variables to be used in future more in depth calculations
    trim_tab_angle = -desired_alpha
    return trim_tab_angle


## Testing sample values:

In [ ]:
apparent_wind_speed = 10  # m/s
wind_direction = 0  # degrees
boat_direction = 0  # degrees
look_up_table = np.array(
    [[50000, 5.75], [100000, 6.75], [200000, 7], [500000, 9.75], [1000000, 10]]
)  # reynolds number, angle of attack

reynolds = compute_reynolds_number(apparent_wind_speed)
alpha = compute_angle_of_attack(reynolds, look_up_table)
trim_tab_angle = compute_trim_tab_angle(alpha, wind_direction, boat_direction)

print("Reynolds number: ", reynolds)
print("Angle of attack: ", alpha)
print("Trim tab angle: ", trim_tab_angle)


## Plot some stuff

In [ ]:
# Reference: https://matplotlib.org/stable/gallery/lines_bars_and_markers/fill_between_demo.html#example-confidence-bands
N = 21
x = np.linspace(0, 10, 11)
y = [3.9, 4.4, 10.8, 10.3, 11.2, 13.1, 14.1, 9.9, 13.9, 15.1, 12.5]

# fit a linear curve and estimate its y-values and their error.
a, b = np.polyfit(x, y, deg=1)
y_est = a * x + b
y_err = x.std() * np.sqrt(1 / len(x) + (x - x.mean()) ** 2 / np.sum((x - x.mean()) ** 2))

fig, ax = plt.subplots()
ax.plot(x, y_est, "-")
ax.fill_between(x, y_est - y_err, y_est + y_err, alpha=0.2)
ax.plot(x, y, "o", color="tab:brown")
